In [2]:
import pandas as pd

from utils.logger import logger

pubmed_search_result = pd.read_csv(
    "../../data/raw_content/pubmed_search_summary_2025_01_03_05_00_54.csv", 
    parse_dates=['Published Date']
)

# pubmed_search_result.describe()

# pubmed_search_result.info()

# pubmed_search_result.head()

pubmed_search_result[0:5]

# pubmed_search_result.

# pubmed_search_result[['PDF URL']]

# logger.info(pubmed_search_result)


/var/folders/_l/b0gq7jr123bdg2xl0t5zgn3c0000gn/T/ipykernel_73992/3306815953.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pubmed_search_result = pd.read_csv(


,Title,PMID,DOI,Authors,Published Date,URL,PDF URL
0,Artificial intelligence and open science in di...,PMC10897520,10.1016/j.xcrm.2023.101379,"Feixiong Cheng, Fei Wang, Jian Tang, Yadi Zhou...",2024 Feb 20/01/01,https://pmc.ncbi.nlm.nih.gov/articles/PMC10897520,https://pmc.ncbi.nlm.nih.gov/articles/PMC10897...
1,Dementia Ideal Care: Ecosystem Map of Best Pra...,PMC11307099,10.3233/JAD-231491,"Wen Dombrowski, Adrienne Mims, Ian Kremer, Ped...",2024 Jun 25/01/01,https://pmc.ncbi.nlm.nih.gov/articles/PMC11307099,https://pmc.ncbi.nlm.nih.gov/articles/PMC11307...
2,Brain aging patterns in a large and diverse co...,PMC11483219,10.1038/s41591-024-03144-x,"Zhijian Yang, Junhao Wen, Guray Erus, Sindhuja...",2024 Aug 15/01/01,https://pmc.ncbi.nlm.nih.gov/articles/PMC11483219,https://pmc.ncbi.nlm.nih.gov/articles/PMC11483...
3,Gene-SGAN: discovering disease subtypes with i...,PMC10774282,10.1038/s41467-023-44271-2,"Zhijian Yang, Junhao Wen, Ahmed Abdulkadir, Yu...",2024 Jan 8/01/01,https://pmc.ncbi.nlm.nih.gov/articles/PMC10774282,https://pmc.ncbi.nlm.nih.gov/articles/PMC10774...
4,10th European Stroke Organisation Conference A...,PMC11157553,10.1177/23969873241245672,NaN,2024 May 15/01/01,https://pmc.ncbi.nlm.nih.gov/articles/PMC11157553,https://pmc.ncbi.nlm.nih.gov/articles/PMC11157...


First we read the data from the csv file and we will use first 10 rows for testing.

Now we will use crawler4ai to scrape the data from the pubmed website. 

According to `URL` column. 

In [8]:
# Download the PDF files from the pubmed website.
import os
from tqdm import tqdm
from utils.pdf_downloader import download_pdf
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered
from marker.settings import settings

from langchain_core.documents import Document

pdf_dir = "../../data/raw_content/pdfs"
markdown_dir = "../../data/raw_content/markdown"
image_dir = "../../data/raw_content/images"

converter = PdfConverter(
    artifact_dict=create_model_dict()
)

for idx, row in tqdm(pubmed_search_result[0:3].iterrows(), desc="Downloading PDFs"):
    """
    Download the PDF file if it does not exist.
    Convert the PDF to markdown and images.
    Save the markdown and images to the corresponding directories.
    """
    
    pmid = row['PMID']
    
    pdf_filename = f"{pmid}.pdf"
    pdf_path = os.path.join(pdf_dir, pdf_filename)
    
    markdown_filename = f"{pmid}.md"
    markdown_path = os.path.join(markdown_dir, markdown_filename)
    
    # Download the PDF file if it does not exist
    if not os.path.exists(pdf_path):
        logger.info(f"Downloading PDF file: {pdf_filename}")
        download_pdf(row['PDF URL'], pdf_path)

    # Convert the PDF to markdown and images
    rendered = converter(filepath=pdf_path)
    text, _, images = text_from_rendered(rendered)

    # Save the markdown and images
    try:
        text = text.encode(settings.OUTPUT_ENCODING, errors='replace').decode(settings.OUTPUT_ENCODING)
        with open(markdown_path, 'w', encoding='utf-8') as f:
            doc = Document(
                page_content=text,
                metadata={
                    "source": row['PDF URL'],
                    "pmid": pmid,
                    "title": row['Title'],
                    "authors": row['Authors'],
                    "published_date": row['Published Date'],
                    "source-tag": "pubmed"
                }
            )
            
            # f.write(doc.model_dump_json())
            
        with open(markdown_path, "w+", encoding=settings.OUTPUT_ENCODING) as f:
            f.write(text)
                
        for img_name, img in images.items():
            img.save(os.path.join(image_dir, f"{pmid}_{img_name}"), settings.OUTPUT_IMAGE_FORMAT)
            
        # with open(os.path.join('../../logs/', f"{pmid}_meta.json"), "w+", encoding=settings.OUTPUT_ENCODING) as f:
        #     f.write(json.dumps(rendered.metadata, indent=2))
                
        logger.info(f'Successfully saved markdown and images for {pmid}')
    except Exception as e:
        logger.error(f'Failed to save markdown and images for {pmid}: {e}')


Loaded layout model datalab-to/surya_layout on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded table recognition model vikp/surya_tablerec on device mps with dtype torch.float16
Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16


100%|██████████| 3/3 [00:00<00:00, 22.49it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing tables: 0it [00:00, ?it/s]


2025-01-15 @ 15:05:30 | INFO     | __main__:<module>:72 - Successfully saved markdown and images for PMC10897520


100%|██████████| 6/6 [00:00<00:00, 39.06it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing tables: 0it [00:00, ?it/s]


2025-01-15 @ 15:05:43 | INFO     | __main__:<module>:72 - Successfully saved markdown and images for PMC11307099


100%|██████████| 7/7 [00:00<00:00, 38.45it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:01<00:00,  1.24s/it]


2025-01-15 @ 15:06:02 | INFO     | __main__:<module>:72 - Successfully saved markdown and images for PMC11483219
